## Brief Projet : 

Vous êtes un data analyst ,vous travaillez en bénévolat pour la médiathèque de la ville qui a mis en place un site web pour la vente de certains livres de son stock.. et elle souhaite analyser les caractéristiques sa clientèle pour mieux comprendre les tendances de leurs reservations/achats de livres. 

La médiathèque vous donne accès à leur site : http://books.toscrape.com/index.html. et souhaite que vous collectez/analysez leurs données.


**Étapes du projet** :

1. **Collecte de données** : 
* Utiliser la bibliothèque `requests` pour envoyer des requêtes HTTP au site web qui répertorie les livres. Vous récupérez le contenu HTML de la page web.

* A l'aide de la bibliothèque `Beautiful Soup`, analyser le contenu HTML du site et extraire les informations pertinentes: parcourir le code HTML, identifier les balises cibles (qui contiennent les données sur les livres, telles que `<div>` ou `<li>` ) et extraire les informations pertinentes telles que le nom du livre, la catégorie, la note moyenne des avis, le nombre de livres en stock, le prix etc.

2. **Nettoyage et préparation des données** : nettoyer les valeurs, convertir les types de données si nécessaire, gérer les valeurs manquantes, etc.


### Importer les librairies

In [3]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Collecte de données

####  Récupérer les livres de la première page

In [48]:
# URL de la page principale
url = "http://books.toscrape.com/"
#url="http://books.toscrape.com/catalogue/page-1.html"
# Envoyer une requête GET à l'URL
response = requests.get(url)

# Vérifier si la requête s'est bien déroulée
if response.status_code == 200:
    # Analyser le contenu HTML de la page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver tous les liens de livres sur la page principale
    book_links = soup.find_all('h3')

    
    # Parcourir les liens de livres
    for link in book_links:
        # Extraire le titre du livre
        title = link.a.attrs['title']
        print(f"Titre : {title}")
      


Titre : A Light in the Attic
Titre : Tipping the Velvet
Titre : Soumission
Titre : Sharp Objects
Titre : Sapiens: A Brief History of Humankind
Titre : The Requiem Red
Titre : The Dirty Little Secrets of Getting Your Dream Job
Titre : The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Titre : The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
Titre : The Black Maria
Titre : Starving Hearts (Triangular Trade Trilogy, #1)
Titre : Shakespeare's Sonnets
Titre : Set Me Free
Titre : Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Titre : Rip it Up and Start Again
Titre : Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
Titre : Olio
Titre : Mesaerion: The Best Science Fiction Stories 1800-1849
Titre : Libertarianism for Beginners
Titre : It's Only the Himalayas


In [5]:
print(link)

<h3><a href="catalogue/its-only-the-himalayas_981/index.html" title="It's Only the Himalayas">It's Only the Himalayas</a></h3>


In [6]:
print(link.find('a').get('title'))

It's Only the Himalayas


In [7]:
print(link.a.attrs['title'])

It's Only the Himalayas


In [8]:
print(link.find('a').get_text())

It's Only the Himalayas


In [9]:
print(len(book_links))
#print(book_links)

20


#### Récupérer les livres de toutes les pages  

In [10]:
url = "http://books.toscrape.com/"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    #livres = soup.find_all(class_='product_pod')
    livres = soup.find_all('article', {'class': 'product_pod'})
    livres_price = soup.find_all("div", {'class': 'product_price'})
    print(livres_price)

[<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>, <div class="product_price">
<p class="price_color">Â£53.74</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>, <div class="product_price">
<p class="price_color">Â£50.10</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>, <div class="product_price">
<p class="price_color">Â£47.82</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    


In [109]:

# Fonction pour extraire les informations d'une page donnée
def extract_book_info(page_url):
    books=[]
    response = requests.get(page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        #livres = soup.find_all(class_='product_pod')
        livres = soup.find_all('article', {'class': 'product_pod'})
        #print(livres)
        for livre in livres:

            titre =  livre.h3.a.get('title')

            rating = livre.p.get("class")[1]

            prix = livre.find("p", {"class":"price_color"}).text[2:]

            disponibilite = livre.find("p",{"class": "instock availability"}).text.strip("\n ")

            books.append({'title': titre, 'rating': rating, 'price': prix,'availability': disponibilite})
            
            df_books=pd.DataFrame(books)
            
            
    return df_books
            
            


In [110]:
p=extract_book_info('http://books.toscrape.com')

In [111]:
p.head()

,title,rating,price,availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.10,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock


In [117]:
p.shape

(20, 4)

In [118]:
find_pages = soup.find("li", {"class": "current"})
end_page = int(find_pages.text.strip().split(" " )[3])
print(end_page)

50


In [119]:
#navigate all the pages based on URL change
base_url = "https://books.toscrape.com/catalogue/page-{}.html"
df_books_ = pd.DataFrame()
all_books_list = []
page_number = 1
while page_number <= end_page:
    page_url = base_url.format(page_number)
    books = extract_book_info(page_url)
    all_books_list.append(books)
    df_books_ = pd.concat([df_books_, books], ignore_index=True)
    page_number += 1


In [120]:
df_books_ #contains all the information from all pages

,title,rating,price,availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.10,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,One,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),Five,16.97,In stock
998,1st to Die (Women's Murder Club #1),One,53.98,In stock


In [116]:
# Une Boocle pour parcourir ttes les pages
#TBD
total_page = soup.find_all("li", {"class": "next"})
print(type(total_page))
total_page

<class 'bs4.element.ResultSet'>


[<li class="next"><a href="catalogue/page-2.html">next</a></li>]

In [45]:

df_books_.shape

(1000, 4)

In [ ]:
df_books_.head(10)

In [47]:
print('All_books:',type(all_books),'de ',len(all_books),'pages,  avec' ,all_books[0].shape[0], 'livres par page')

All_books: <class 'list'> de  50 pages,  avec 20 livres par page


#### Récupérer les livres de toutes les pages avec leurs  catégories

In [169]:
# Function to get category URLs
def get_category_urls():
    base_url = "https://books.toscrape.com/"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    category_section = soup.find('ul', {'class': 'nav nav-list'})
    category_links = category_section.find_all('a')
    category_urls = {}
    for link in category_links:
        category_name = link.text.strip()
        category_url = base_url + link.get('href')
        if category_name != 'Books':
            category_urls[category_name] = category_url.replace('index.html', 'page-{}.html')
    return category_urls

# Get all category URLs
category_urls = get_category_urls()
category_urls

{'Travel': 'https://books.toscrape.com/catalogue/category/books/travel_2/page-{}.html',
 'Mystery': 'https://books.toscrape.com/catalogue/category/books/mystery_3/page-{}.html',
 'Historical Fiction': 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-{}.html',
 'Sequential Art': 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-{}.html',
 'Classics': 'https://books.toscrape.com/catalogue/category/books/classics_6/page-{}.html',
 'Philosophy': 'https://books.toscrape.com/catalogue/category/books/philosophy_7/page-{}.html',
 'Romance': 'https://books.toscrape.com/catalogue/category/books/romance_8/page-{}.html',
 'Womens Fiction': 'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/page-{}.html',
 'Fiction': 'https://books.toscrape.com/catalogue/category/books/fiction_10/page-{}.html',
 'Childrens': 'https://books.toscrape.com/catalogue/category/books/childrens_11/page-{}.html',
 'Religion': 'https://books.toscrape.c

In [171]:
page_number = 1
response = requests.get('https://books.toscrape.com/catalogue/category/books/crime_51/index.html')
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    find_pages = soup.find("li", {"class": "current"})
    # end_page = int(find_pages.text.strip().split(" " )[3])
print(find_pages)

None


In [179]:
# Function to get all pages of books for a given category

def get_all_books_for_category(base_url):
    
    all_books_df = pd.DataFrame()
    page_number = 1
    end_page = 1
    page_url = base_url.format(page_number)
    response = requests.get(page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        find_pages = soup.find("li", {"class": "current"})
        if find_pages is not None:
            end_page = int(find_pages.text.strip().split(" " )[3])
        else:
            end_page = 1
    print(page_url)
    while page_number <= end_page:
        page_url = base_url.format(page_number)
        books_df = extract_book_info(page_url)
        all_books_df = pd.concat([all_books_df, books_df], ignore_index=True)
        
        page_number += 1
        
    return all_books_df

df = get_all_books_for_category('https://books.toscrape.com/catalogue/category/books/erotica_50/page-{}.html')
print(df)

https://books.toscrape.com/catalogue/category/books/erotica_50/page-1.html


UnboundLocalError: cannot access local variable 'df_books' where it is not associated with a value

In [167]:
# Extract book information from all categories
all_books_df = pd.DataFrame()
for category, url in category_urls.items():
    category_books_df = get_all_books_for_category(url)
    category_books_df['category'] = category  # Add category column
    all_books_df = pd.concat([all_books_df, category_books_df], ignore_index=True)
    print(all_books_df)

Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []
Empty DataFrame
Columns: [category]
Index: []


KeyboardInterrupt: 

In [168]:
# Filter rows where the category is "Travel"
travel_books_df = all_books_df[all_books_df['category'] == 'crime']
travel_books_df

,category


In [148]:
print(all_books_df["category"].value_counts())

category
Default               160
Nonfiction            120
Add a comment          80
Sequential Art         80
Fiction                80
Fantasy                60
Young Adult            60
Food and Drink         40
Mystery                40
Childrens              40
Romance                40
Historical Fiction     40
Poetry                 19
Classics               19
History                18
Womens Fiction         17
Horror                 17
Science Fiction        16
Science                14
Music                  13
Business               12
Travel                 11
Thriller               11
Philosophy             11
Humor                  10
Autobiography           9
Art                     8
Religion                7
Psychology              7
Spirituality            6
New Adult               6
Christian Fiction       6
Self Help               5
Sports and Games        5
Biography               5
Health                  4
Contemporary            3
Christian               3
Pol

In [271]:
print(len(categories), 'categories \n', 'répartie comme suit:', df_categories)

51 categories 
 répartie comme suit:               category  nb_books
0                Books      1020
1               Travel        11
2              Mystery        52
3   Historical Fiction        46
4       Sequential Art        95
5             Classics        19
6           Philosophy        11
7              Romance        55
8       Womens Fiction        17
9              Fiction        85
10           Childrens        49
11            Religion         7
12          Nonfiction       130
13               Music        13
14             Default       172
15     Science Fiction        16
16    Sports and Games         5
17       Add a comment        87
18             Fantasy        68
19           New Adult         6
20         Young Adult        74
21             Science        14
22              Poetry        19
23          Paranormal         1
24                 Art         8
25          Psychology         7
26       Autobiography         9
27           Parenting         1
28    

In [279]:
#Save data into csv file
book_infos.to_csv('books.csv',index=False,header=True)

### Nettoyage et préparation des données

In [307]:
df1=pd.read_csv('books.csv')
df1.head()

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Soumission,One,50.10,In stock,Books
3,Sharp Objects,Four,47.82,In stock,Books
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books


#### Comprendre la structure des données + Analyse descriptive 


* Etudier la structure du DataFrame : head(), info(),describe() , observer les diff types de données, les valeurs manquantes,les doublons, les statistiques sommaires, etc.


In [1]:
#TBD

##### Gestion des valeurs manquantes

* Rechercher des valeurs manquantes  à l'aide de la méthode `.isnull()` ou `.isna()`. 

=> Si vous en trouvez, vous pouvez décider de les supprimer ou de les remplacer par des valeurs appropriées.

In [2]:
 #Vérifier les valeurs manquantes
#TBD

#####  Gestion des doublons

* Utiliser la méthode `drop_duplicates()` pour supprimer les lignes dupliquées.


In [3]:
# Vérifier les doublons dans le DataFrame
#TBD

#### Conversion de types de données

* Vérifier que les colonnes "rating" et "price", availability ont les types de données appropriés. 



In [ ]:
#TBD

### Stockage de données 

* Proposer une modélisation cohérente et créer la BDD dans votre SGBDR

### Analyse exploratoire et Visualisation des données

* faire une analyse exploratoire des données pour mieux comprendre la distribution des valeurs dans chaque colonne, identifier des tendances ou des valeurs aberrantes potentielles..

#### Analyse univariée, Visualiser les données

* Distribution de la variable 'price' =>  Création d'un histogramme pour le prix

In [4]:
#TBD


* Distribution de la variable 'availability' 

In [5]:
#TBD

* Distribution de la variable 'rating'

In [6]:
#TBD

#### Analyser les categories de livres 

* comparer la catégorie 'Books' avec les reste des categories ..


In [ ]:
#TBD



* Création d'un diagramme à barres pour la répartition des livres par catégorie


In [7]:
#TBD

#### Identifier les valeurs aberrantes (outliers)

* Création d'un boxplot pour le prix par catégorie


In [8]:
#TBD

## Bonus:

###  Proposer d'autres axes d'analyse

### continuer le scraping du site en récupérant plus d'informations sur les livres..